In [26]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [52]:
categories = ['EDUCATION', 'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 'BEAUTY', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 
       'SOCIAL', 'SHOPPING', 
       'PRODUCTIVITY', 'PHOTOGRAPHY', 
       'MEDICAL', 'PARENTING', 
       'EDUCATION', 'BEAUTY',
       'COMMUNICATION', 'TOOLS']

In [28]:
def get_links(data):
    '''Given html data this will extract all the links'''
    link_list = []
    for link in data.findAll('a'):
        get_link = link.get('href')
        link_list.append(get_link)
    return link_list

In [29]:
def webpage_html(url):
    r = requests.get(url)
    html = r.text
    return html

In [49]:
def google_scraper(categories):
    '''Will cycle through every category and then request the site, pass the soup to 
        get_links and then clean the list and append results to a new list in a dict
        format that has the format of {cat_name : [link list]}'''
    dicty = dict()
    for category in categories:
        url = f'https://play.google.com/store/apps/category/{category}/collection/topselling_free'
        html = webpage_html(url)
        soup = BeautifulSoup(html)
        links = get_links(soup)
        del links[0:107]
        clean = list(set(links))
        new_list = [app for app in clean if app.find('details') != -1]
        dicty[category] = new_list
        time.sleep(3)
    return dicty


In [38]:
cat = categories

In [39]:
stuff = google_scraper(cat)

In [44]:
stuff1 = stuff[1]

In [51]:
for key, value in stuff1.items():
    print(len(value))

60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
0
60


In [3]:
dictionary = [{'cats' : [1,2,3]}, {'dogs' : [10,11,12]}]

In [48]:
x = 
for k,v in stuff1.items():
    print(v)

['/store/apps/details?id=com.app.shakealertla', '/store/apps/details?id=com.funpop.android_hatch', '/store/apps/details?id=com.imperinca.downloadmusicforfreetomyphonefastguide', '/store/apps/details?id=air.com.rosettastone.mobile.CoursePlayer', '/store/apps/details?id=com.vitotechnology.StarWalk2Free', '/store/apps/details?id=com.blackboard.android.bbstudent', '/store/apps/details?id=curso.d.ingles.gratis', '/store/apps/details?id=com.getmimo', '/store/apps/details?id=com.wonder', '/store/apps/details?id=dollsforkids.coloringbook.fun', '/store/apps/details?id=net.appbeta.deepwebmobile', '/store/apps/details?id=com.memrise.android.memrisecompanion', '/store/apps/details?id=com.fws.plantsnap2', '/store/apps/details?id=com.pablo.gallegoFalcon.PDLNormalES', '/store/apps/details?id=com.quizlet.quizletandroid', '/store/apps/details?id=com.duolingo', '/store/apps/details?id=org.pbskids.video', '/store/apps/details?id=com.yousician.yousician', '/store/apps/details?id=com.draw.ai.learn.to.draw'

In [ ]:
def get_description(category_list):
    url = 'https://play.google.com'
    for category,  in category_list.items():
        category = 
        